In [ ]:
# LCEL = nouvelle syntaxe pour rendre plus facile l'appel et la liaison de chaînes et d'agents

# Méthodes classiques attachées aux chaînes : 
# invoke = à partir d'un input
# stream = à partir d'un input, on renvoie une réponse
# batch = à partir d'une liste d'inputs

In [ ]:
# Set up environment
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
#!pip install pydantic==1.10.8

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
# Le composant ci-dessous prend un message de chat et le transforme en string (simple!)
from langchain.schema.output_parser import StrOutputParser

In [ ]:
# Simple chain

In [ ]:
# On crée un prompt template, on initialise le modèle et l'output parser
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [ ]:
# On crée cette chaîne (syntaxe de type Linux)
chain = prompt | model | output_parser

In [ ]:
# On appelle la chaîne, avec en input un dico contenant les éléments nécessaires (ici : "topic" dans le prompt template)
chain.invoke({"topic": "bears"})

In [ ]:
# More complex chain : utiliser des docs annexes pour améliorer la génération de textes
# Runnable Map pour envoyer les inputs de l'usager vers le prompt

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
# On initialise le retriever
# Le vectorstore est composé de quelques phrases à peine
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [ ]:
# Avec la fonction get_relevant_documents,
# on obtient les documents les plus pertinents associés à une question, dans l'ordre de pertinence
retriever.get_relevant_documents("where did harrison work?")

In [ ]:
retriever.get_relevant_documents("what do bears like to eat")

In [ ]:
# Dans cette chaîne, les prompts suivront toujours le même template ci-dessous
# Il y aura deux inputs pour le prompt : 'context' et 'question'
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# Le composant "RunnableMap" part d'un dictionnaire en input et retourne un autre dictionnaire, 
# au format souhaité par le composant suivant (prompt)
from langchain.schema.runnable import RunnableMap

In [ ]:
# Le contexte en input du prompt consiste en l'ensemble des documents pertinents de notre documentation,
# repérés par le retriever
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [ ]:
chain.invoke({"question": "where did harrison work?"})

In [ ]:
# Pour voir en détail ce que fait le composant "RunnableMap"
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [ ]:
inputs.invoke({"question": "where did harrison work?"})

In [ ]:
# Bind and OpenAI functions

In [ ]:
# Exemple de fonction d'Open AI
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [ ]:
# Syntaxe pour que le modèle de chat intègre la fonction ci-dessus
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [ ]:
runnable = prompt | model

In [ ]:
# La réponse est au format de sortie du modèle
# On constate que 'content' est vide et 
# qu'il y a appel de la bonne fonction (dans les additional_kwargs, 
# avec récupération du bon code d'aéroport
runnable.invoke({"input": "what is the weather in sf"})

In [ ]:
# On ajoute d'autres fonctions
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [ ]:
# On met à jour le modèle
model = model.bind(functions=functions)

In [ ]:
# On met à jour la chaîne
runnable = prompt | model

In [ ]:
# Le modèle a bien reconnu la bonne fonction dans l'exemple ci-dessous
runnable.invoke({"input": "how did the patriots do yesterday?"})

In [ ]:
# Fallbacks
# C'est une sorte d'auto-réparation
# Cas d'usage ci-dessous : demander à un modèle de sortir un JSON en output

In [ ]:
# Les llms de cette library sont un peu moins bons que les ChatModels utilisés jusqu'ici dans ce notebook
from langchain.llms import OpenAI
import json

In [ ]:
# text-davinci est un vieux modèle d'OpenAI
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="text-davinci-001"
)
# Chaine simple pour sortir un JSON
simple_chain = simple_model | json.loads

In [ ]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [ ]:
# L'ouput n'a pas vraiment une structure de JSON
simple_model.invoke(challenge)

In [ ]:
# La preuve : json.loads ne le comprend pas
simple_chain.invoke(challenge)

In [ ]:
# Les nouveaux modèles d'OpenAI sont bons pour sortir des formats de JSON valides
# Il faut juste ajouter un parser pour convertir l'output du ChatModel en string (le texte du message)
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [ ]:
chain.invoke(challenge)

In [ ]:
# Fallback : pour switcher d'une première chaîne à une liste de chaînes de secours en cas d'erreur de la première
final_chain = simple_chain.with_fallbacks([chain])

In [ ]:
final_chain.invoke(challenge)

In [ ]:
# Interface

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [ ]:
chain.invoke({"topic": "bears"})

In [ ]:
# Batch : on met en input une liste d'inputs
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

In [ ]:
# Stream : on fait apparaître les outputs les uns après les autres
# Utiles pour avoir les premiers éléments de réponse quand les suivants tardent à venir
for t in chain.stream({"topic": "bears"}):
    print(t)

In [ ]:
# Méthodes asynchronisées (d'où le await)
response = await chain.ainvoke({"topic": "bears"})
response